In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import regex
import re
import seaborn as sn
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from textblob import Word
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score,roc_curve,auc
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.metrics import classification_report
import fastText
plt.style.use('ggplot')
%matplotlib notebook

In [2]:
def fonction_nettoyage_text(df):
    # supprission des ponctuations
    rm_ponct = str.maketrans('','',string.punctuation)
    df = df.apply(lambda x:x.translate(rm_ponct))
    # suppression les unicodes
    df = df.apply(lambda x:x.encode("ascii","ignore").decode("utf-8"))
                                                
    # suppression des URLs
    df = df.apply(lambda x:re.sub(r'http\S+',"",x))
    
    # suppression des stopwords
    stop_en = stopwords.words("english")
    df = df.apply(lambda x:" ".join(x.lower() for x in np.str(x).split() if x.lower() not in stop_en))
                                                  # Lemmatisation
    df = df.apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
                  
    return(df)

In [3]:
# import des données
dataset = pd.read_csv("data_set_version_final.csv")
dataset.resume=fonction_nettoyage_text(dataset.resume)

## Split Train Test:

In [4]:
X_train,X_test,Y_train,y_test = train_test_split(dataset.resume,dataset.Labels,test_size=0.3,shuffle=True)
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.3,shuffle=True)

###  Exporter les données train /test/val:

In [5]:
x_train.to_csv("FastTestFolder/xtrain.txt",index=False)
x_val.to_csv("FastTestFolder/xval.txt",index=False)
X_test.to_csv("FastTestFolder/Xtest.txt",index=False)

/home/tantely/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/home/tantely/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
/home/tantely/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avo

In [ ]:
model =fastText.train_supervised(
        input="FastTestFolder/xtrain.txt", epoch=25, lr=1.0, wordNgrams=2, verbose=2, minCount=1
)

In [ ]:
from pyfasttext import FastText
model = FastText()
model.skipgram(input='FastTestFolder/xtrain.txt', output='model', epoch=10, lr=0.3)


In [6]:
from pyfasttext import FastText
model = FastText()
model.cbow(input='FastTestFolder/xtrain.txt', output='model', epoch=10, lr=0.3,dim=300)


In [10]:
res_machine=model.get_numpy_sentence_vector("machine")

In [ ]:
from pyfasttext import FastText
import numpy as np
class FastWord2Vec:
    def __init__(self,inputFile,outputFile,epoch,lr,dim):
        self.inputFile=inputFile
        self.outputFile=outputFile
        self.epoch=epoch
        self.dim=dim
        self.lr=lr
        global model
        
    def __average_word(self,X,modelWords):
        np.array([
            np.mean([self.get_numpy_sentence_vector(w) for w in words.split() if w in modelWords]or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

    def fit(self,X,y=None):
        X.to_csv(self.inputFile,index=False)
        self.model=FastText().cbow(input=self.inputFile,output=self.outputFile,epoch=self.epoch,lr=self.lr,dim=self.dim)
        return self
    def transform(self,X,y):
        return self.__average_word(X,self.words)        
        

In [ ]:
cbowTransformer=FastWord2Vec(inputFile='FastTestFolder/xtrain.txt',outputFile='model',epoch=10,lr=0.5,dim=100)
cbowTransformer.fit(x_train)

/home/tantely/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
